In [52]:
from targets import create_topological_fasta
import argparse
import sys
from rich.console import Console
import os
import tempfile
from multiprocessing import cpu_count
from Bio import SeqIO
import re
import subprocess
import pandas as pd
from io import StringIO
import re
import codecs

In [53]:
# consider https://github.com/google/python-fire for command line interface
class Args:
    genome_file = "GCA_000005845.2.gb"
    pam = "NGG"
    barcode_length = 20
    orientation = 'reverse'
    pam_direction = 'downstream'
    omit_intergenic = True
    omit_offtargets = True
    omit_ambiguous = False
    mismatches = 1
    keep_top = 10
    tile_size = barcode_length
    full_overlap = True

args = Args()

In [54]:
ESCAPE_SEQUENCE_RE = re.compile(r'''
    ( \\U........      # 8-digit hex escapes
    | \\u....          # 4-digit hex escapes
    | \\x..            # 2-digit hex escapes
    | \\[0-7]{1,3}     # Octal escapes
    | \\N\{[^}]+\}     # Unicode characters by name
    | \\[\\'"abfnrtv]  # Single-character escapes
    )''', re.UNICODE | re.VERBOSE)

def decode_escapes(s):
    def decode_match(match):
        return codecs.decode(match.group(0), 'unicode-escape')

    return ESCAPE_SEQUENCE_RE.sub(decode_match, s)

In [55]:

def is_dna(sequence):
    return all(base in 'GATC' for base in sequence)

def find_sequences_with_barcode_and_pam(topological_fasta_file_name, barcode_length, pam):
    matching_sequences = set()
    pam_regex = re.compile(pam.replace('N', '[ATGC]'))

    with open(topological_fasta_file_name, "rt") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            # Consider both the original sequence and its reverse complement
            for sequence in [str(record.seq), str(record.seq.reverse_complement())]:
                for i in range(len(sequence) - barcode_length - len(pam) + 1):
                    # If PAM is downstream
                    if args.pam_direction == 'downstream' and pam_regex.match(sequence[i+barcode_length:i+barcode_length+len(pam)]):
                        spacer = sequence[i:i+barcode_length]
                        if is_dna(spacer):
                            matching_sequences.add(spacer)
                    # If PAM is upstream
                    elif args.pam_direction == 'upstream' and pam_regex.match(sequence[i-len(pam):i]):
                        spacer = sequence[i:i+barcode_length]
                        if is_dna(spacer):
                            matching_sequences.add(spacer)

    return matching_sequences

# create a sgRNA fasta file such as >sequence\nsequence\n
def create_sgRNA_fasta(matching_sequences, sgRNA_fasta_file_name):
    with open(sgRNA_fasta_file_name, "wt") as handle:
        for sequence in matching_sequences:
            handle.write(f">{sequence}\n{sequence}\n")


In [56]:
def main(args):
    num_threads = cpu_count() // 2
    
    console=Console(file=sys.stderr)
    
    console.log(f"Settings: {args}")

    
    with tempfile.TemporaryDirectory() as working_dir:
        console.log("[bold red]Initializing barcode target builder[/bold red]")
        console.log(f"Stay tuned... running 'targets.py' to find guides for {args.genome_file} with {args.barcode_length}bp barcodes and {args.pam} PAM sequence")

        topological_fasta_file_name = os.path.join(working_dir, os.path.splitext(os.path.basename(args.genome_file))[0] + ".fasta")

        sgRNA_fasta_file_name = os.path.join(working_dir, "sgRNA.fasta")
    
        create_topological_fasta(args.genome_file, topological_fasta_file_name)
    
        matching_sequences = find_sequences_with_barcode_and_pam(topological_fasta_file_name, args.barcode_length, args.pam)
        
        create_sgRNA_fasta(matching_sequences, sgRNA_fasta_file_name)
        
        result = subprocess.run([
            "python", 
            "targets.py", 
            sgRNA_fasta_file_name, 
            args.genome_file, 
            args.pam,
            "--pam_direction", args.pam_direction,
            str(args.mismatches)
            ], stdout=subprocess.PIPE, text=True, check=True)

        targets = pd.read_csv(StringIO(result.stdout), sep='\t')    
        
        console.log(f"Found {len(targets):,} guides")
        console.log(f"Removing guides based on settings")
        
        targets['target'] = targets['target'].str.upper()
        
        if args.orientation == 'forward':
            # omit everything where sp_dir and tar_dir are not the same
            targets = targets.loc[targets['sp_dir'] == targets['tar_dir']]
        
        elif args.orientation == 'reverse':
            # omit everything where sp_dir and tar_dir are the same
            targets = targets.loc[targets['sp_dir'] != targets['tar_dir']]

        if args.omit_offtargets:

            console.log("[bold red]Removing guides with off-targets[/bold red]")
            len_before = len(targets)
            # Extract the number of sites from the 'note' column
            targets.loc[:, 'sites'] = targets['note'].str.extract(r'(\d+) site', expand=False).astype(int)            
            # Create a mask that is True for rows where 'sites' is 1
            mask = (targets['sites'] == 1)

            # Apply the mask to the DataFrame
            targets = targets[mask]
            console.log(f"Removed {(len_before - len(targets)):,} guides")
        # remove everything where mismatches > 0
        if args.mismatches > 0:
            console.log("[bold red]Removing guides with mismatches. There shouldn't be any![/bold red]")
            len_before = len(targets)
            
            targets = targets.loc[targets['mismatches'] == 0]
            
            console.log(f"Removed {(len_before - len(targets)):,} guides")
        
        if args.omit_ambiguous:
            console.log("[bold red]Removing ambiguous guides, this will be a lot![/bold red]")

            # Extract the number of sites, genes, and intergenic regions from the 'note' column
            targets['sites'] = targets['note'].str.extract(r'(\d+) site', expand=False).astype(int)
            targets['genes'] = targets['note'].str.extract(r'(\d+) gene', expand=False).astype(int)
            targets['intergenic'] = targets['note'].str.extract(r'(\d+) intergenic', expand=False).astype(int)

            # Create a mask that is True for rows where 'sites', 'genes', and 'intergenic' are 1
            mask = (targets['sites'] == 1) & (targets['genes'] == 1) & (targets['intergenic'] == 1)

            len_before = len(targets)

            # Apply the mask to the DataFrame
            targets = targets[mask]

            console.log(f"Removed {(len_before - len(targets)):,} guides")

        if args.omit_intergenic:
            console.log("[bold red]Removing intergenic regions[/bold red]")

            # Create a mask that is True for rows where 'note' does not contain "intergenic"
            mask = ~targets['note'].str.contains('intergenic')

            len_before = len(targets)

            # Apply the mask to the DataFrame
            targets = targets[mask]

            console.log(f"Removed {(len_before - len(targets)):,} guides")

        
        if args.full_overlap:
            console.log("[bold red]Removing guides that don't fully overlap with the gene[/bold red]")
            # Sort the DataFrame by 'target', 'spacer' and 'locus_tag'
            
            len_before = len(targets)

            # Create a new DataFrame where 'overlap' is equal to args.barcode_length
            overlap_df = targets.loc[targets['overlap'] == args.barcode_length]
            
            # Extract the 'spacer' column from overlap_df and convert it to a set
            overlap_spacers = set(overlap_df['spacer'])

            # Create a mask that is True for rows where 'spacer' is in overlap_spacers
            mask = targets['spacer'].isin(overlap_spacers)

            # Apply the mask to the DataFrame
            targets = targets[mask]
            
            console.log(f"Removed {(len_before - len(targets)):,} guides")
        
        # If tile_size was not provided, set it to the value of barcode_length
        if args.tile_size is None:
            args.tile_size = args.barcode_length
            
            # Sort the DataFrame by 'locus_tag' and 'offset'
            targets = targets.sort_values(['locus_tag', 'offset'])

            # Group the DataFrame by 'locus_tag'
            grouped = targets.groupby('locus_tag')

            # Initialize a set to store the selected spacers
            selected_spacers = set()

            # Iterate over each group
            for name, group in grouped:
                last_offset = group['offset'].min() - args.tile_size
                for index, row in group.iterrows():
                    # If the current offset is at least tile_size away from the last offset, add the spacer to the set
                    if row['offset'] >= last_offset + args.tile_size:
                        selected_spacers.add(row['spacer'])
                        last_offset = row['offset']

            # Create a mask that is True for rows where 'spacer' is in selected_spacers
            mask = targets['spacer'].isin(selected_spacers)

            # Apply the mask to the DataFrame
            targets = targets[mask]
                        
            
        if args.keep_top:
            console.log(f"[bold red]Keeping only the top {args.keep_top} guides for each gene[/bold red]")
            
            len_before = len(targets)
        
            # Sort the DataFrame by 'locus_tag', 'offset', and 'overlap'
            targets = targets.sort_values(['locus_tag', 'offset', 'overlap'])

            # Group the DataFrame by 'locus_tag' and keep the top n rows
            top_targets = targets.groupby('locus_tag').head(args.keep_top)

            # Get the spacers that are in the top n rows for each gene
            top_spacers = top_targets['spacer'].unique()

            # Filter the original DataFrame to keep only the rows that are in the top n rows for each gene or that target multiple genes
            targets = targets[targets['spacer'].isin(top_spacers)]
            
            console.log(f"Removed {(len_before - len(targets)):,} guides")
               
    # Convert all numeric columns to integers
    targets = targets.apply(lambda col: pd.to_numeric(col, errors='coerce').fillna(0).astype(int) if col.dtypes != object else col)
    
    targets = targets.sort_values(['chr', 'tar_start', 'tar_end', 'locus_tag', 'offset', 'overlap'])
    
    # print csv to output
    targets.to_csv(sys.stdout, sep='\t', index=False, na_rep='None')



In [57]:
main(args)

[08:55:18] Settings: <__main__.Args object at 0x795377f68590>                                        ]8;id=82024;file:///tmp/ipykernel_3927844/198950525.py\198950525.py]8;;\:]8;id=354006;file:///tmp/ipykernel_3927844/198950525.py#6\6]8;;\

           Initializing barcode target builder                                                      ]8;id=634149;file:///tmp/ipykernel_3927844/198950525.py\198950525.py]8;;\:]8;id=791834;file:///tmp/ipykernel_3927844/198950525.py#10\10]8;;\

           Stay tuned... running 'targets.py' to find guides for GCA_000005845.2.gb with 20bp       ]8;id=974281;file:///tmp/ipykernel_3927844/198950525.py\198950525.py]8;;\:]8;id=678219;file:///tmp/ipykernel_3927844/198950525.py#11\11]8;;\
           barcodes and NGG PAM sequence                                                                           

[08:55:24] Initializing barcode target seeker                     ]8;id=58277;file:///home/ryandward/Git/barcoder/targets.py\targets.py]8;;\:]8;id=670639;file:///home/ryandward/Git/barcoder/targets.py#452\452]8;;\
           Annotating regions to identify...                      ]8;id=699187;file:///home/ryandward/Git/barcoder/targets.py\targets.py]8;;\:]8;id=864182;file:///home/ryandward/Git/barcoder/targets.py#460\460]8;;\
[08:55:29] Generating topological coordinate maps...              ]8;id=138137;file:///home/ryandward/Git/barcoder/targets.py\targets.py]8;;\:]8;id=301199;file:///home/ryandward/Git/barcoder/targets.py#474\474]8;;\
[08:55:37] Aligning annotations to genome...                      ]8;id=68044;file:///home/ryandward/Git/barcoder/targets.py\targets.py]8;;\:]8;id=782376;file:///home/ryandward/Git/barcoder/targets.py#479\479]8;;\
[08:55:59] Finding matches...                                     ]8;id=471723;file:///home/ryandward/Git/bar